In [1]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

### Queries

In [2]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

In [3]:
%%time

# count of MG enabled LUs at wire center level
# 10/24/22 - add UNI ecd date
# 10/25/22 - add N. number per Jen request

mg_query = """

            SELECT  distinct
                    O.WIRE_CENTER_ID
                    ,O.UNI_POLYGONID
                    ,O.ADDRESS
                    ,O.HNO
                    ,O.H_SUFFIX
                    ,O.H_PRE
                    ,O.ST
                    ,O.ST_SUFFIX
                    ,O.ST_POST
                    ,O.UNIT
                    ,O.COM
                    ,O.STATE
                    ,O.ZIP_CODE
                    ,O.LU_LATITUDE
                    ,O.LU_LONGITUDE
                    ,O.UNI_ECD
                    ,O.UNI_NDSJOBNUMBER
                    ,O.NDSJOBID

                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in ('1178832','1178694','1178144',
                                          '1179498','1179547','1179121','1179499',  --Phoenix AZ turned up 9/30/22
                                          '1179687','1179674', -- Golden CO 10/10/22
                                          '1180012', -- portland OR turned up 10/15/22
                                          '1179864', -- denver CO turned up 10/15/22
                                          '1179569','1179553','1179688', -- st paul and mn turned up 10/14/22
                                          '1179808', -- turned up 10/21/22
                                          '1179862','1179836','1179604','1179838','1179887','1180054','1180222', --turned up 10/21/22
                                          '1179157','1179574', -- turned up 10/21/22
                                          '1179976', -- turned up 10/27/22
                                          '1179583', '1180254', '1180028', -- turned up 10/28/22
                                          '1179774','1180265','1180107','1180306','1180026', -- turned up 11/4/22
                                          '1180288', -- turned up 11/7/22
                                          
                                          --waiting turn up
                                          '1180067', -- PHNXAZSO
                                          '1180386', -- ARVDCOMA
                                          '1179849', -- MPLSMNBE
                                          '1180344', -- MPLSMNBE
                                          '1179924'  --STPLMNBE


-- seattle locations not ready until after 11/18
--'1180327',
--'1180111',
--'1180086',
--'1180117'


                                          
                                          ) --add in MG polygon IDs here


"""

mg_lu_ora = pd.read_sql(mg_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 297 ms
Wall time: 25.2 s


In [6]:
# export

import time

TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_mg_polygon_addresses.xlsx"

with pd.ExcelWriter(excelfilename) as writer:
    mg_lu_ora.to_excel(writer, sheet_name='main addresses', index = False)
#     df_1179808.to_excel(writer, sheet_name='1179808', index = False)